<a href="https://colab.research.google.com/github/alinat2/MLINDIV/blob/main/Sspan_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symmetry Span Data Processing

**Author**: Alina Tu

**Contact**: alinat2@uci.edu

**Last Updated**: 8/15/2022

**About**: This preprocessing script is for the Symmetry Span (Sspan) data files of the Individual Differences and Robotics (IndivRobotics) project. Symmetry refers to the grid used when determining whether the design is symmetrical (SymmetryDisplays: symm number). Matrix refers to the red squares, recalled in sequential order (MatrixID: matrix number).

In [ ]:
!pip install rpy2==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext rpy2.ipython

In [ ]:
# Getting started [Packages and working directory]
%%R
install.packages("rprime")
library(rprime)
library(dplyr)
library(tidyverse)

(as ‘lib’ is unspecified)







	‘/tmp/Rtmp853XTb/downloaded_packages’

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union



✔ tibble  3.1.8     ✔ stringr 1.4.1
✔ tidyr   1.3.0     ✔ forcats 1.0.0
✔ readr   2.1.4     

✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Input the working directory containing the Sspan Eprime data files
# Each file is named "SspanShort-" + "subject ID" + "session number" (i.e., SspanShort-111-1.txt)
%%R
workdir <- "/content/drive/Shareddrives/IndividualDifferencesUCIUCSB/RAFolder/Raw Data/Session 4.2/Symmetry Span"
setwd(workdir)


In [ ]:
# Create a function to organize Eprime data
%%R
read <- function(file_name) {
  # Read in a text file generated by Eprime
  file_lines <- read_eprime(file_name)
  # Convert lines from an Eprime file into EprimeFrame objects
  file_frames <- FrameList(file_lines)
  # Make it a data frame
  file_df <- to_data_frame(file_frames)
  # Clean up data
  data <- as_tibble(file_df)
  # data[data == "?"] <- "NA"
}

#test_data <- read("SspanShort-805-4.txt")

In [ ]:
# Store list of names for all Eprime data files
%%R
files <- dir(pattern = "*.txt")

trial_full <- tibble()
recall_full <- tibble()
participant_full <- tibble()

for (indiv_eprime_datafile in files){
  readable_data <- read(indiv_eprime_datafile)
  sub_id <- as.numeric(readable_data$Subject)[1]
  
  # Symmetry-matrix trial sequences
  trial_data <- readable_data[readable_data$Procedure == "trialdo" | readable_data$Procedure == "RedrawProc", ]
  trial_proc <- as_tibble(trial_data) %>% 
    select(Procedure, ShowSymm.RT, CheckResponse.RT, CollectClick.RT)
    # ShowSymm.RT: RT taken to click past the symmetry grid
    # CheckResponse.RT: RT taken to select a true/false response
    # CollectClick.RT: RT taken to select a red square during recall
  trial_proc <- cbind(rep(sub_id, nrow(trial_proc)), trial_proc)
  trial_full <- rbind(trial_full, trial_proc)
  
  # Overall test "recall" trials
  recall_data <- readable_data[readable_data$Procedure == "recall", ]
  recall_proc <- as_tibble(recall_data)[3:8,] %>% # there are 6 test trials
    select(Cycle, Sample, SpanScore, SpanTotal)
  recall_proc <- cbind(rep(sub_id, nrow(recall_proc)), recall_proc)
  recall_full <- rbind(recall_full, recall_proc)
  
  # Overall scores
  summary_data <- readable_data[readable_data$Procedure == "SessionProc", ]
  summary_proc <- as_tibble(summary_data)[1,] %>%
    select(SymmErrorTotal, SpeedErrorTotal, AccErrorTotal, SspanScore, SspanTotal)
    # Sspan Score: sum of SpanScore in each recall_proc procedure (# of correctly recalled red squares)
    # note: set size = list length
    # Sspan Total: sum of SpanTotal in each recall_proc procedure (total possible red squares to recall)
  avg_ShowSymm.RT <- mean(as.numeric(trial_proc$ShowSymm.RT), na.rm = TRUE)
  avg_CheckResponse.RT <- mean(as.numeric(trial_proc$CheckResponse.RT), na.rm = TRUE)
  avg_CollectClick.RT <- mean(as.numeric(trial_proc$CollectClick.RT), na.rm = TRUE)
  summary_proc <- cbind(sub_id, avg_ShowSymm.RT, avg_CheckResponse.RT, avg_CollectClick.RT, summary_proc)
  participant_full <- rbind(participant_full, summary_proc)

  # 1. (later) pull RTs per trial thing
}

In [ ]:
%%R
# print(participant_full)
colnames(trial_full) <- c("sub_id", "ShowSymm.RT", "CheckResponse.RT", "CollectClick.RT")
colnames(recall_full) <- c("sub_id", "Cycle", "Sample", "SpanScore", "SpanTotal")

print("Writing CSVs...")
path_out <- "/content/drive/Shareddrives/IndividualDifferencesUCIUCSB/RAFolder/DataProcessingScripts/Session 4.2/Symmetry Span"
write.csv(trial_full, file.path(path_out, "Sspan_trial_full.csv"), row.names=FALSE)
write.csv(recall_full, file.path(path_out, "Sspan_recall_full.csv"), row.names=FALSE)
write.csv(participant_full, file.path(path_out, "Sspan_participant_full.csv"), row.names=FALSE)
print("CSVs have been created.")